In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

# Importing the dataset
df = pd.read_csv('/kaggle/input/berlin-airbnb-data/listings_summary.csv')


In [ ]:
df.columns

Let's drop the columns that are not necessary

In [ ]:
df.head()

In [ ]:
df

In [ ]:
df.isnull().sum()

In [ ]:
df.info()

In [ ]:
# let's drop  the unnesessary columns, it can vary and may sometime impact our 
# accuracy also, so be cautious while ddoing so
df.drop(['listing_url', 'scrape_id', 'last_scraped', 'experiences_offered', 'neighborhood_overview',
        'transit', 'access', 'interaction', 'house_rules',
       'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url',
       'host_about', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location',
       'host_acceptance_rate', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_listings_count',
       'host_total_listings_count', 'host_verifications',
       'host_has_profile_pic', 'host_identity_verified', 'street',
       'neighbourhood', 'neighbourhood_cleansed', 'host_is_superhost',
       'city', 'state', 'zipcode', 'market', 'weekly_price', 'monthly_price', 
       'smart_location', 'country_code', 'country','calendar_updated', 'has_availability',
       'availability_30', 'availability_60', 'availability_90', 'instant_bookable',
       'availability_365', 'calendar_last_scraped', 'number_of_reviews', 'is_location_exact',
       'first_review', 'last_review', 'requires_license','maximum_nights',
       'license', 'jurisdiction_names', 'require_guest_profile_picture', 'require_guest_phone_verification',
       'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness',
       'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value',
       'calculated_host_listings_count', 'reviews_per_month', 'is_business_travel_ready', 'minimum_nights'],
        axis=1, inplace=True)

In [ ]:
# Checking whether there is any repeated or duplicate values
df.duplicated().sum()

Above output indicates that there is no duplicate values in our dataset after deleting the unnecessary columns

In [ ]:
# Checking the missing values 
df.isnull().sum()

In [ ]:
# Setting 'id' as an index
df = df.set_index('id')

In [ ]:
# Since above we found that there are many missing values in some columns
# so dropping the columns with extremely high missing values
df.drop(['space', 'notes', 'square_feet', 'host_response_time', 'host_response_rate'],
       axis = 1, inplace = True)

In [ ]:
df.isna().sum()

In [ ]:
# Still we can see that there is many missing values in some columns so now
# we will fill some of these missing values of columns
# So lets replace the NaNs in bathrooms and bedrooms with 1 
df.bathrooms.fillna(1, inplace = True)
df.bedrooms.fillna(1, inplace = True)

# we can replace the NaNa in beds by neighbour column 'accomodate'
#df.beds.fillna(df['accomodates'], inplace = True)


In [ ]:
df.head()

In [ ]:
df['beds'].isnull().sum()

In [ ]:
#we can replace the NaNa in beds by neighbour column 'accomodate'
df.beds.fillna(df['accommodates'], inplace = True)

In [ ]:
# Communities deployment 
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize = (10,5))
sns.countplot(y = df['neighbourhood_group_cleansed'], order = df.neighbourhood_group_cleansed.value_counts().index)
plt.xlabel("Quantity of listings", fontsize = 'medium')
plt.ylabel('')
plt.title("Communities deployment", fontsize = 'large')

In [ ]:
df['neighbourhood_group_cleansed']

In [ ]:
df.neighbourhood_group_cleansed.value_counts().index

In [ ]:
# Property type deployment - TOP-10 types
plt.figure(figsize = (15,5))
sns.countplot(df['property_type'], order = df.property_type.value_counts().iloc[:10].index)
plt.xlabel("")
plt.ylabel("Quantity of listings", fontsize = 'large')
plt.title("Property type")

In [ ]:
# Room type deployment
plt.figure(figsize = (5,5))
sns.countplot(df['room_type'], order = df.room_type.value_counts(normalize = True).index)

**'price' and other money related columns**

In [ ]:
# Cleaning (replace '$') and formatig price-related columns
df.price = df.price.str.replace('$', '').str.replace(',', '').astype(float)
#df.security_deposit = df.security_deposit.str.replace('$', '').str.replace(',', '').astype(float)


In [ ]:
df.security_deposit = df.security_deposit.str.replace('$', '').str.replace(',', '').astype(float)


In [ ]:
df.cleaning_fee = df.cleaning_fee.str.replace('$', '').str.replace(',', '').astype(float)
df.extra_people = df.extra_people.str.replace('$', '').str.replace(',', '').astype(float)

# NaNs in security_deposit and cleaning_fee seem to be 0
#df.security_deposit.filna(0, inplace = True)
#df.cleaning_fee.fillna(0, inplace = True)


In [ ]:
# As we already have seen that there is NaNs in security_deposit and cleaning_fee
# so lets fill them
df.security_deposit.fillna(0, inplace = True)
df.cleaning_fee.fillna(0, inplace = True)


In [ ]:
df['security_deposit'].isnull().sum()

In [ ]:
df['cleaning_fee'].isna().sum()

so we can see that there is no NaNs in cleaning_fee and security deposit now

In [ ]:
# Checking suspiciously low prices
print(df[(['price', 'name'])][df.price<10])

In [ ]:
# Dropping rows with price < 8$
df = df.drop(df[df.price<8].index)

In [ ]:
# Now cheecking the price greater than $8 but less than 10
print(df[(['price', 'name'])][df.price<10])

In [ ]:
# Checking suspiciously high prices 
df['price'].plot(kind = 'box', xlim = (0,600),vert = False, figsize = (16,1))

In [ ]:
# Since high price are not  affordable for everyone so dropping the extremely 
# high price

df = df.drop(df[df.price > 380].index)

In [ ]:
df.isnull().sum()

In [ ]:
print(df[(['price', 'name'])][(df.price<80) & (df.price>20)])

**Extracting and Working out data re room size**

In [ ]:
# Extract number that may contain info re square of rooms from 'description' 
#columns (contains, 's/m/S/M')
df['room_size'] = df['description'].str.extract("(\d{2,3}\s[smSM])", expand = True)
df['room_size'] = df['room_size'].str.replace("\D", "").astype(float)
rv = len(df) - df['room_size'].isna().sum()
print('Real values in "room_size" column:   ', rv)
print('Real values in "room_size" column (%):   ', round(rv/len(df)*100, 1), '%')


# (C) This cell of code was taken from the original research, done by Britta Bettendorf

In [ ]:
# Extract numbers that may contain info re square of rooms from 'name' columns
# (contains 's/m/S/M')

df['room_size_name'] = df['name'].str.extract("(\d{2,3}\s[smSM])", expand = True)
df['room_size_name'] = df['room_size_name'].str.replace("\D", "").astype(float)

rv = len(df) - df['room_size_name'].isna().sum()
print('Real values in "room_size_name" column:    ', rv)
print('Real values in "room_size_name" column(%):    ', round(rv/len(df)*100, 1), '%')

In [ ]:
df.room_size.fillna(0, inplace = True)

In [ ]:
# Updatig column 'room_size' with values extracted from column 'name'
df.loc[df['room_size'] == 0, 'room_size'] = df['room_size_name']

In [ ]:
# We don't needit any more
df.drop(['room_size_name'], axis = 1, inplace = True)

In [ ]:
# Checking suspiciously low sizes
print(df[(['room_size', 'name'])][(df.room_size < 10)])

In [ ]:
# Dropping rows with  suspiciously low sizes
df = df.drop(df[df.room_size < 10].index)

In [ ]:
# Checking suspiciously high sizes
df['room_size'].plot(kind = 'box', vert = False, figsize = (16,1))

In [ ]:
print(df[(['room_size', 'name'])][df.room_size > 250])

In [ ]:
# Dropping values of suspiciously high sizes
df.loc[df['room_size'] > 250, 'room_size'] = ''
df.room_size.replace(to_replace = '', value = np.nan, inplace = True)

In [ ]:
# Wehave NaN's in our column, 2/3 of all values
df.room_size.isna().sum()

In [ ]:
df.isna().sum()

In [ ]:
# New df for further regression
df_temp = df[['neighbourhood_group_cleansed', 'accommodates', 'bathrooms', 'bedrooms',
             'beds', 'price', 'security_deposit', 'cleaning_fee', 'guests_included',
             'extra_people', 'room_size']]

In [ ]:
print(df_temp.shape)
df_temp.head(10).transpose()

In [ ]:
# Taking care of categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder_X = LabelEncoder()
categorical_cols = ['neighbourhood_group_cleansed']
df_temp[categorical_cols] = df_temp[categorical_cols].apply(lambda col: labelencoder_X.fit_transform(col.astype(str)))
df_temp.head(10).transpose()

In [ ]:
df['neighbourhood_group_cleansed'].unique().sum()

In [ ]:
# Arranging datasets by existence of 'room_size' value

train_set = df_temp[df_temp['room_size'].notnull()]
test_set = df_temp[df_temp['room_size'].isnull()]

# Arranging X-taining and X-testing datasets
X_train = train_set.drop('room_size', axis = 1)
X_test = test_set.drop('room_size', axis = 1)

# Arranging y-training datasets 
y_train = train_set['room_size']

In [ ]:
# Regression Model
from sklearn.ensemble import RandomForestRegressor
regressor = RandomForestRegressor(n_estimators = 300, random_state = 123)
regressor.fit(X_train, y_train)

# Predicting the Test set results
y_pred = regressor.predict(X_test)

In [ ]:
y_pred

In [ ]:
# Introduction of predicted data to the main dataset 'df'
y_pred = pd.DataFrame(y_pred)
y_pred.columns = ['room_size']

In [ ]:
temp_id = pd.DataFrame(X_test.index)
temp_id.columns = ['temp_id']

y_pred = pd.concat([y_pred, temp_id], axis = 1)
y_pred.set_index(['temp_id'], inplace = True)

df_pred = pd.concat([X_test, y_pred], axis = 1)
df_pred.head()

In [ ]:
df_pred.shape

In [ ]:
train_set.shape

In [ ]:
df_temp = pd.DataFrame()
df_temp = pd.concat([df_pred, train_set], axis = 0)
print(df_temp.shape)
df_temp.head().transpose()

In [ ]:
df_temp.head(10)

In [ ]:
df_temp.head(10).transpose()

In [ ]:
X_test

In [ ]:
X_train

In [ ]:
y_pred

In [ ]:
train_set

In [ ]:
test_set

In [ ]:
df_temp

In [ ]:
y_pred

In [ ]:
# Checking again suspiciously low sizes
print(df_temp[(['room_size'])][(df_temp.room_size<10)])

In [ ]:
# Checking suspiciously high sizes
df_temp['room_size'].plot(kind = 'box', vert = False, figsize = (16,1))

In [ ]:
print(df.shape)

In [ ]:
df.head()

In [ ]:
df.head(2).transpose()

In [ ]:
print(df_temp.shape)
df_temp.head().transpose()

In [ ]:
df = df[['property_type', 'amenities', 'cancellation_policy']]
print(df.shape)
df.isna().sum()

In [ ]:
df = pd.concat([df, df_temp], axis = 1)
print(df.shape)
df.head(3).transpose()

In [ ]:
# Checking whether there is a null value or not in df
df.isna().sum()

In [ ]:
import tensorflow as tf

**Amenities score introduction**

In [ ]:
# Let's explore amenities
pd.set_option('display.max_colwidth', -1)
df.amenities.head(5)

In [ ]:
# Let's introduce new column with score of amenities
df['amen_score'] = df['amenities'].str.count(',') + 1

In [ ]:
# We don't need it any more
df.drop(['amenities'], axis = 1, inplace = True)

In [ ]:
df['amen_score']

In [ ]:
df.head().transpose()

In [ ]:
df.isna().sum()

*Every columns is now non empty*

**Gradient Boosting**

In [ ]:
# A separate copy for TF
df_tf = df.copy()

In [ ]:
# Taking care of categorical data
from sklearn.preprocessing import LabelEncoder
labelencoder_X = LabelEncoder()
categorical_cols = ['property_type', 'cancellation_policy']
df[categorical_cols] = df[categorical_cols].apply(lambda col: labelencoder_X.fit_transform(col.astype(str)))
df.head(10).transpose()

In [ ]:
# Creating DV and IV sets
X = df.drop('price', axis = 1)
y = df['price']

# Splitting the dataset into the training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.25, random_state = 123)

In [ ]:
# Gradient Boosting Regression
from sklearn.ensemble import GradientBoostingRegressor
regressor = GradientBoostingRegressor(n_estimators = 100, max_depth = 3, min_samples_split = 2,
                                      learning_rate = 0.1)
regressor.fit(X_train, y_train)

# Predicting the test set results
y_pred = regressor.predict(X_test)

# Finding the mean_sqaured error (MSE)
from sklearn.metrics import mean_squared_error
mse = mean_squared_error(y_test, y_pred)

# Finding the r2 score or the variance (R2)
from sklearn.metrics import r2_score
r2 = r2_score(y_test, y_pred)

# Applying k-fold Cross Validation
from sklearn.model_selection import cross_val_score
accuracies = cross_val_score(estimator = regressor, X = X_test, y = y_test, cv = 10)

# Printing metrics 
print("RMSE Error:", round(np.sqrt(mse), 2))
print("R2 Score:", round(r2, 4))
print("Mean Accuracy:", round(accuracies.mean(), 2))
print("Std Deviation:", round(accuracies.std(), 4))

**Tensor Flow DNN_Regressor**

In [ ]:
import tensorflow as tf

In [ ]:
# Creating DV and IV sets
X_tf = df_tf.drop('price', axis = 1)
y_tf = df_tf['price']

# Splitting the datasets into the Training set and Test set

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tf, y_tf, test_size = 0.25,
                                                   random_state = 123)

In [ ]:
# Feature columns
property_type = tf.feature_column.categorical_column_with_hash_bucket('property_type', hash_bucket_size=50)
cancellation_policy = tf.feature_column.categorical_column_with_hash_bucket('cancellation_policy', hash_bucket_size=10)
neighbourhood_group_cleansed = tf.feature_column.numeric_column('neighbourhood_group_cleansed')
accommodates = tf.feature_column.numeric_column('accommodates')
bathrooms = tf.feature_column.numeric_column('bathrooms')
bedrooms = tf.feature_column.numeric_column('bedrooms')
beds = tf.feature_column.numeric_column('beds')
security_deposit = tf.feature_column.numeric_column('security_deposit')
cleaning_fee = tf.feature_column.numeric_column('cleaning_fee')
guests_included = tf.feature_column.numeric_column('guests_included')
room_size = tf.feature_column.numeric_column('room_size')
amen_score = tf.feature_column.numeric_column('amen_score')

In [ ]:
emb_property_type = tf.feature_column.embedding_column(property_type, dimension = 33)
emb_cancellation_policy = tf.feature_column.embedding_column(cancellation_policy, dimension = 5)

In [ ]:
feat_cols = [emb_property_type, emb_cancellation_policy, neighbourhood_group_cleansed, accommodates, bathrooms,
            bedrooms, beds, security_deposit, cleaning_fee, guests_included, room_size, amen_score]


In [ ]:
# Input function
from tensorflow_core.estimator import inputs
input_func =  tf.compat.v1.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=100,num_epochs=1000,shuffle=True)

In [ ]:
# Creating and training model
model = tf.estimator.DNNRegressor(hidden_units = [12,12,12], feature_columns = feat_cols)


In [ ]:
model.train(input_fn = input_func, steps = 1000)

In [ ]:
pred_input_func = tf.compat.v1.estimator.inputs.pandas_input_fn(x = X_test, batch_size = 10, num_epochs = 1, shuffle = False)

In [ ]:
predictions = list(model.predict(pred_input_func))

In [ ]:
y_pred = []
for i in predictions:
    y_pred.append(i['predictions'][0])

In [ ]:
from sklearn.metrics import mean_squared_error
tf_mse = mean_squared_error(y_test, y_pred)
print("MSE Error:", round(tf_mse, 2))
print("RMSE Error:", round(np.sqrt(tf_mse), 2))